### N Gram In N Gram Analysis

In [4]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [5]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [6]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# # adding native word to shared word
# twogram_start = 0  # 0 native word start index
# twogram_end = 200  # 28 native word end index

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False
sixgram_sentence_check = False
sevengram_sentence_check = False
eightgram_sentence_check = False
ninegram_sentence_check = False
tengram_sentence_check = False

# n gram sample
threegram_sample = 6
fourgram_sample = 4
fivegram_sample = 2

## n gram select
#twogram_select_start = 0
#twogram_select_end = 10000
#
#threegram_select_start = 0
#threegram_select_end = 10000
#
#fourgram_select_start = 0
#fourgram_select_end = 10000
#
#fivegram_select_start = 0
#fivegram_select_end = 10000

# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [7]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/5-N Gram In N Gram Analysis/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [8]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [9]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [13]:
#df_word.to_excel("Word_Selected_45000.xlsx", index=False)

In [14]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [15]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [16]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

200

In [14]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]

df_twogram

In [15]:
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)

# all word should be used at least once
#df_twogram_select = df_merge_result2.iloc[twogram_start:twogram_end,]

df_merge_result2

In [16]:
# word usage in twogram condition
word_num_dict = {}
for i in word_list:
    word_num_dict[f"{i}"] = 0

result_list_select = []
var_list = []
for i in range(len(df_merge_result2)):
    twogram = df_merge_result2.loc[i,"twogram"]
    frequency = df_merge_result2.loc[i,"frequency"]
    words = word_tokenize(twogram)   
    temp_list = [word for word in words]
    temp_list = temp_list + var_list
    # word count for max
    dict_list_count = Counter(temp_list)
    count_list = list(dict_list_count.values())
    # word count for min
    count_list2 = list(word_num_dict.values())

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        pass
    else:
        var_list = temp_list
        result_list_select.append([twogram,frequency]) 

        for item2 in dict_list_count.items(): 
            word_num_dict[item2[0]] = item2[1]        

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}")

In [19]:
df_twogram_select = pd.DataFrame(result_list_select, columns=["twogram","frequency"])
df_twogram_select.sort_values(by="frequency", ascending=False, inplace=True)
df_twogram_select.reset_index(drop=True, inplace=True)
df_twogram_select

In [24]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [25]:
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
2215200,ve beynini uçurdu,10
2215201,generale asla bu,10
2215202,çekmiş bana değil,10
2215203,benim söylediklerimi dinlemedin,10


In [28]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [29]:
d_list4  = df_fourgram.iloc[:,0].values.tolist()

resultlist4 = []
manager = multiprocessing.Manager()
resultlist4 = manager.list()

def word_in_wordgroup(d_list4):
    mergelist = []
    try:
        word = d_list4.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist4.append(d_list4)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list4) # string_word liste

result_list4 = list(resultlist4)
df_result4 = pd.DataFrame(result_list4, columns=[0])  # add columns parameter for empty result
df_result4 = df_result4.rename(columns = {0: "fourgram"})
df_merge4 = pd.merge(df_result4, df_fourgram, how="left", on="fourgram")
df_merge_result4 = df_merge4.sort_values(by="frequency", ascending=False)
df_merge_result4.drop_duplicates(inplace=True)
df_merge_result4.reset_index(drop=True, inplace=True)
df_fourgram_select = df_merge_result4
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
1937931,o yeni kız afrikadan,7
1937932,orada dikilip vücuduma bakıyorlar,7
1937933,olsaydık tam on bin,7
1937934,neyse umarım bu daha,7


In [32]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [33]:
d_list5  = df_fivegram.iloc[:,0].values.tolist()

resultlist5 = []
manager = multiprocessing.Manager()
resultlist5 = manager.list()

def word_in_wordgroup(d_list5):
    mergelist = []
    try:
        word = d_list5.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist5.append(d_list5)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list5) # string_word liste

result_list5 = list(resultlist5)
df_result5 = pd.DataFrame(result_list5, columns=[0])  # add columns parameter for empty result
df_result5 = df_result5.rename(columns = {0: "fivegram"})
df_merge5 = pd.merge(df_result5, df_fivegram, how="left", on="fivegram")
df_merge_result5 = df_merge5.sort_values(by="frequency", ascending=False)
df_merge_result5.drop_duplicates(inplace=True)
df_merge_result5.reset_index(drop=True, inplace=True)
df_fivegram_select = df_merge_result5
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
597091,evet son üç saattir neredeydim,10
597092,seninle yaşamak için geldiğimiz ilk,10
597093,olunca ne yapar biliyor musun,10
597094,zamanı gelince seni babana göndereceğim,10


In [ ]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram_select, df_fourgram_select, df_fivegram_select], axis=1)
df_ngram_concat

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup_simple(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup_simple(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [ ]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [ ]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [ ]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

In [ ]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="right", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

In [ ]:
df_ngram_sample_join_merge.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Join_Result.xlsx", index=False)

#### Adding Frequency For Ratio

In [ ]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_three_freq

In [ ]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

In [ ]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

In [ ]:
df_ngram_sample_concat = pd.concat([df_twogram_select,df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

In [ ]:
df_ngram_sample_concat.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Frequency_Result.xlsx", index=False)

#### Calculate Result And Select Ratio

In [ ]:
df_ngram_threegram_unique = df_ngram_sample_concat[["threegram","three_freq"]].drop_duplicates()
df_ngram_fourgram_unique = df_ngram_sample_concat[["fourgram","four_freq"]].drop_duplicates()
df_ngram_fivegram_unique = df_ngram_sample_concat[["fivegram","five_freq"]].drop_duplicates()

In [ ]:
twogram_result_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_threegram_unique["three_freq"].sum()
fourgram_result_freq = df_ngram_fourgram_unique["four_freq"].sum()
fivegram_result_freq = df_ngram_fivegram_unique["five_freq"].sum()

In [ ]:
twogram_select_freq = df_twogram.iloc[twogram_select_start:twogram_select_end,]["frequency"].sum()
threegram_select_freq = df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()
fourgram_select_freq = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]["frequency"].sum()
fivegram_select_freq = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]["frequency"].sum()

In [ ]:
(twogram_result_freq/twogram_select_freq)*100

In [ ]:
(threegram_result_freq/threegram_select_freq)*100

In [ ]:
(fourgram_result_freq/fourgram_select_freq)*100

In [ ]:
(fivegram_result_freq/fivegram_select_freq)*100

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_*_Result.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [17]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [18]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [19]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 4

In [20]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [21]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [22]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [23]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [24]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [25]:
word_list = df_word_select["word"].to_list()

In [26]:
df_file = pd.read_excel("Talk Time 2 Video List.xlsx")
df_file

,2Gram,3Gram,4Gram
0,şu anda,bir şey yok,en son ne zaman
1,ya da,bir şey değil,bir şey var mı
2,tabii ki,şu ana kadar,bir şey mi oldu
3,o zaman,öyle değil mi,bir şey daha var
4,iyi olur,çok özür dilerim,seni görmek çok güzel
5,hiçbir şey,yok bir şey,çok güzel değil mi
6,gördün mü,tamam o zaman,bunu daha önce de
7,daha fazla,sen iyi misin,bana yardım eder misin
8,bu harika,neden bu kadar,o kadar basit değil
9,bir dakika,ne işin var,bu da ne demek


In [28]:
df_file_count = word_count_result(df_file,["2Gram","3Gram","4Gram"])
df_file_count

,word,word_count
0,ne,9
1,şey,9
2,bir,9
3,bu,8
4,değil,7
...,...,...
96,harika,1
97,hadi,1
98,görmek,1
99,gördün,1


In [29]:
word_set = set(word_list)
file_set = set(df_file_count["word"].to_list())

In [30]:
file_set.difference(word_set)

{'acele',
 'ana',
 'anda',
 'basit',
 'dikkat',
 'dilerim',
 'eder',
 'emin',
 'etme',
 'fikrim',
 'gidelim',
 'gidiyorsun',
 'gördün',
 'görmek',
 'hem',
 'hoşça',
 'işin',
 'kal',
 'kendine',
 'memnun',
 'merak',
 'miyim',
 'nereye',
 'neyin',
 'oldum',
 'sağ',
 'tanıyor',
 'yardımcı',
 'yeter',
 'yolunda'}

In [ ]:
df_two_three = pd.read_excel("/home/kurubal/Downloads/Twogram Threegram.xlsx")
df_two_three

In [ ]:
word_count_result(df_two_three,["threegram"])

In [ ]:
df_two_three["twogram"].count()

In [ ]:
df_two_three["threegram"].count()

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_two_three, "twogram","threegram",threegram_sample)
df_two_in_three

In [ ]:
set_all_twogram = set(df_two_three["twogram"].dropna().to_list())
set_twogram_in_threegram = set(df_two_in_three["twogram"].to_list())

In [ ]:
set_all_twogram.difference(set_twogram_in_threegram)

In [ ]:
set_threegram_cover_twogram = set(df_two_in_three["threegram"].to_list())
df_selected_threegram = pd.DataFrame(set_threegram_cover_twogram,columns=["threegram"])
df_selected_threegram

In [ ]:
df_selected_threegram.to_excel("Threegram_Selected.xlsx", index=False)

In [ ]:
word_count_result(df_selected_threegram,["threegram"])

In [ ]:
df_threegram_word_usage_select = word_usage_result(word_list, df_selected_threegram, "threegram", word_use_num_min, word_use_num_max)
df_threegram_word_usage_select

In [ ]:
word_count_result(df_threegram_word_usage_select,["threegram"])